<a href="https://colab.research.google.com/github/Sushmitha-Katti/Artshaala--Frontend/blob/master/CIFAR10_RESNET18n_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIFAR 10 data trained on RESNET 18 MODEL

In [ ]:
import os
import time
import math

import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm



# --------------Training Method-------------------------------------------
def train(model, device, train_loader, criterion, optimizer, epoch):
    """
    trains the neural network once for complete dataset. Adds the Accracy and Loss to the metrics dictionary
    Args: 
        model: neural network
        device: cuda or cpu
        train_loader: Train Dataset Loader
        criterion: Loss Function
        optimizer: Optimiser function
        echo: Number for logging
    """
    train_loss = 0.0
    correct = 0
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _,pred = torch.max(outputs, dim=1)
        correct += torch.sum(pred==target).item()
        pbar.set_description(desc= f'epoch = {epoch} loss={loss.item()} batch_id={batch_idx}')
    metrics['train_accuracy'].append(100 * correct / len(train_loader.dataset))
    metrics['train_loss'].append(train_loss/len(train_loader))

# -------------- Testing Method ------------------------------------------------
def test(model, device, test_loader, criterion):
    """
    tests the neural network once for complete dataset. Adds the total Accracy, Loss, classwise accuracy to the metrics dictionary.
    Args:
        model: neural network
        device: cuda or cpu
        train_loader: Train Dataset Loader
        criterion: Loss Function
        optimizer: Optimiser function
        echo: Number for logging
    """
    model.eval()
    test_loss = 0
    correct = 0
    class_correct = {i: {'correct': 0, 'total': 0}  for i in range(len(class_names))}
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            test_loss += criterion(outputs, target).item()  # sum up batch loss
            _,pred = torch.max(outputs, dim=1)
            correct += torch.sum(pred==target).item()
            for i in class_correct:
                class_correct[i]['correct'] += torch.sum((pred == target) * (target == i)).item()
                class_correct[i]['total'] += torch.sum((target == i)).item()

    test_loss /= len(test_loader)
    metrics['test_accuracy'].append(100 * correct / len(test_loader.dataset))
    metrics['test_loss'].append(test_loss)

    # Class Wise Accuracy
    for key, value in class_label_maping.items():
        _acc = class_correct[value]['correct']/class_correct[value]['total'] * 100
        metrics[key].append(_acc)

    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.2f}%)')


# -------------- Main Train Model ------------------------------------------------------------

def train_model(model, criterion, optimizer, num_epochs=2, checkpoint=None):
    """
    A main method which calls both train and test method for each epoch and saves the best model
    Args:
        model: neural network
        device: cuda or cpu
        train_loader: Train Dataset Loader
        criterion: Loss Function
        optimizer: Optimiser function
        echo: Number for logging
    """
    since = time.time()
    if checkpoint is None:
        best_loss = math.inf
    else:
        print(f'Val loss: {checkpoint["best_loss"]}')
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        best_loss = checkpoint['best_loss']

    for epoch in range(num_epochs):
        metrics['epoch'].append(epoch + 1)
        train(model, device, trainloader, criterion, optimizer, epoch)
        test(model, device, testloader, criterion)
        # Check if the current model has less loss than best model and update accordingly
        if metrics['test_loss'][-1] < best_loss:
            best_loss = metrics['test_loss'][-1]
            torch.save(model.state_dict(), CHECK_POINT_PATH)

    time_elapsed = time.time() - since
    print(f"Training is Complete: Time taken is {time_elapsed//60}min {time_elapsed%60}sec")

# -------------------- Write Training Summary Markdown file -------------------------------------------------------
def write_summary():
    summary = "<h1 align='center'>Training Summary</h1>\n"
    summary += "\n"
    summary += "## Data\n"
    data_params = [[
        'Total Train Data Set', len(trainset)
        ],
        [
            'Total Test Data Set', len(testset)
        ],
        [
            'Total Classes', len(class_names)
        ],
        [
            'Class Names', ", ".join(class_names)
        ],
    ]
    summary += pd.DataFrame(data_params, columns=['Description', 'value']).to_markdown()
    summary += "\n"
    summary += "## Model\n"
    model_params = [[
        'Model ', 'Resnet 18(Pretrained)'
        ],
        [
            'Optimiser', 'SGD'
        ],
        [
            'Loss function ', 'Cross Entropy(lr=0.001, momentum=0.9)'
        ],
        [
            'Class Names', ", ".join(class_names)
        ],
    ]
    summary += pd.DataFrame(model_params, columns=['Description', 'value']).to_markdown()
    summary += "\n"
    summary += "## Training\n"
    summary += metrics_df.to_markdown()
    summary += "\n"

    summary += "## Graphs\n"
    plt.plot(metrics['epoch'], metrics['train_loss'], label = "Train Loss")
    plt.plot(metrics['epoch'], metrics['test_loss'], label = "Test Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Loss Graph")
    plt.legend()
    plt.savefig('loss.jpg')
    plt.close()
    plt.plot(metrics['epoch'], metrics['train_accuracy'], label = "Train Accuracy")
    plt.plot(metrics['epoch'], metrics['test_accuracy'], label = "Test Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title("Accuracy Graph")
    plt.legend()
    plt.savefig('accuracy.jpg')
    plt.close()
    plt.plot(metrics['epoch'], metrics[class_names[0]], label = f"Class Wise Accuracy of {class_names[0]}")
    plt.plot(metrics['epoch'], metrics[class_names[1]], label = f"Class Wise Accuracy of {class_names[1]}")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title("Class Wise Accuracy Graph")
    plt.legend()
    plt.savefig('class_wise.jpg')

    summary += "![](loss.jpg)"
    summary += "\n"
    summary += "![](accuracy.jpg)"
    summary += "\n"
    summary += "\n"
    summary += "![](class_wise.jpg)"
    summary += "\n"

    with open("Summary.md", "w") as f:
        f.write(summary)

In [ ]:
CHECK_POINT_PATH = 'cifar10_resnet18.pth'
METRICS_FILE = 'metrics.csv'
cuda = torch.cuda.is_available()
SEED = 1

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

Batch_size = 64
if cuda:
  Batch_size = 128

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=Batch_size, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=Batch_size)

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, **dataloader_args)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, **dataloader_args)




  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Files already downloaded and verified


In [ ]:
trainloader.dataset.class_to_idx

{'airplane': 0,
 'automobile': 1,
 'bird': 2,
 'cat': 3,
 'deer': 4,
 'dog': 5,
 'frog': 6,
 'horse': 7,
 'ship': 8,
 'truck': 9}

In [ ]:




class_names = trainset.classes
class_label_maping = trainset.class_to_idx

print(class_names)
print(f'Train image size: {trainset}')
print(f'Validation image size: {testset}')
metrics = {'epoch': [], "train_accuracy": [], "train_loss": [], "test_accuracy": [], "test_loss": []}

# Columns for class wise accuracy
for i in class_names:
    metrics[i] = []

model = torchvision.models.resnet18(pretrained=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
# for param in model.parameters():
#     param.requires_grad = True
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))
model = model.to(device)

try:
    checkpoint = torch.load(CHECK_POINT_PATH)
    print("checkpoint loaded")
except Exception:
    checkpoint = None
    print("checkpoint not found")


train_model(model,
            criterion,
            optimizer,
            num_epochs=30,
            checkpoint=checkpoint)

metrics_df = pd.DataFrame(metrics)
metrics_df.to_csv(METRICS_FILE, index=False)


['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
Train image size: Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )
Validation image size: Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

cuda:0
checkpoint not found


  0%|          | 0/391 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
epoch = 0 loss=0.8554606437683105 batch_id=390: 100%|██████████| 391/391 [02:08<00:00,  3.04it/s]


...


Test set: Average loss: 0.9884, Accuracy: 7991/10000 (79.91%)


epoch = 28 loss=0.22156639397144318 batch_id=390: 100%|██████████| 391/391 [02:08<00:00,  3.05it/s]



Test set: Average loss: 0.9778, Accuracy: 7990/10000 (79.90%)


epoch = 29 loss=0.042525820434093475 batch_id=390: 100%|██████████| 391/391 [02:09<00:00,  3.03it/s]



Test set: Average loss: 0.9728, Accuracy: 7980/10000 (79.80%)
Training is Complete: Time taken is 66.0min 15.760053396224976sec
